In [6]:
import sys, subprocess
print("Interpreter:", sys.executable)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "flirpy", "exifread"])


Interpreter: c:\ProgramData\anaconda3\python.exe


0

In [1]:
import importlib, inspect
import flirpy
print("flirpy:", getattr(flirpy, "__version__", "unknown"), " @ ", inspect.getfile(flirpy))
print(importlib.util.find_spec("flirpy.io.thermal_image"))


flirpy: unknown  @  C:\Users\boxwa\AppData\Roaming\Python\Python312\site-packages\flirpy\__init__.py
None


In [ ]:
# meta_from_thermal.py
import json
from pathlib import Path
from flirpy.io.thermal_image import ThermalImage
import numpy as np
from PIL import Image

def load_temp_map_c(flir_path: str) -> np.ndarray:
    """Return per-pixel temperatures in °C from a FLIR radiometric JPEG/TIFF."""
    try:
        from flirpy.io.thermal_image import ThermalImage
    except ImportError as e:
        raise SystemExit("Please pip install flirpy") from e
    ti = ThermalImage(flir_path)
    temp = ti.get_temperature()  # °C
    return np.asarray(temp, dtype=np.float32)

def build_meta(
    flir_path: str,
    out_json: str,
    scale: str = "max",                # "max" or "min"
    mask_png: str | None = None,       # optional binary mask: 255=keep, 0=ignore
    firstN: int = 50                   # how many tie coordinates to store
) -> dict:
    T = load_temp_map_c(flir_path)
    H, W = T.shape

    # Apply mask if provided (keep=255)
    if mask_png:
        M = Image.open(mask_png).convert("L").resize((W, H), Image.NEAREST)
        keep = np.array(M) > 0
    else:
        keep = np.ones_like(T, dtype=bool)

    if not np.any(keep):
        raise ValueError("Mask removed all pixels—nothing left to score.")

    Tmasked = np.where(keep, T, np.nan)
    tmin = float(np.nanmin(Tmasked))
    tmax = float(np.nanmax(Tmasked))

    if scale == "max":
        target_val = tmax
        ys, xs = np.where(Tmasked == tmax)
    elif scale == "min":
        target_val = tmin
        ys, xs = np.where(Tmasked == tmin)
    else:
        raise ValueError("scale must be 'max' or 'min'.")

    if len(xs) == 0:
        raise RuntimeError("No target pixel found after masking.")

    # Choose a canonical target (first extreme found, top-left by np.where order)
    x0, y0 = int(xs[0]), int(ys[0])

    # Collect tie coordinates as [x, y] lists
    ties_xy = [[int(x), int(y)] for y, x in zip(ys, xs)]
    ties_firstN = ties_xy[:firstN]

    meta = {
        "width": int(W),
        "height": int(H),
        "target": {"x": x0, "y": y0, "tempC": round(float(target_val), 2)},
        "ties": {"count": int(len(ties_xy)), "firstN": ties_firstN},
        "score": {"dtMax": round(tmax - tmin, 2), "scale": scale},
    }

    Path(out_json).write_text(json.dumps(meta, indent=2))
    return meta

# ---------- example usage ----------
# Set these paths:
THERMAL_PATH = "assets\\puzzles\\01\\IMG_1_MRT.png"                 # your radiometric FLIR image
MASK_PATH    = "assets\\puzzles\\01\\IMG_1_mask.png"     # optional: the 'no-sky' mask you saved
OUT_JSON     = "assets\\puzzles\\01\\meta.json"

if __name__ == "__main__":
    meta = build_meta(THERMAL_PATH, OUT_JSON, scale="max", mask_png=MASK_PATH, firstN=25)
    print("Wrote:", OUT_JSON)
    print(json.dumps(meta, indent=2))


: 